In [1]:
#!pip install hana_ml --upgrade
#!pip install hdfs
#!pip install sklearn

In [2]:
#import hana_ml
#print(hana_ml.__version__)

In [3]:
#from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import pickle
import io
import csv
import pandas as pd
from io import StringIO
from math import sqrt
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

In [4]:
from hdfs import InsecureClient
client = InsecureClient('http://datalake:50070')

In [5]:
with client.read('/shared/ml/data/CREDIT_RISK_ANALYSIS/Loan_Data.csv') as reader:
    df_data = pd.read_csv(reader, sep=',')
df_data.head(5)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Application_Type
0,LP001002,Male,No,0,Graduate,No,5849,0.0,146.412162,360,1.0,Urban,Y,INDIVIDUAL
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.000000,360,1.0,Rural,N,INDIVIDUAL
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.000000,360,1.0,Urban,Y,INDIVIDUAL
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.000000,360,1.0,Urban,Y,INDIVIDUAL
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.000000,360,1.0,Urban,Y,INDIVIDUAL


In [6]:
df1 = df_data

In [7]:
#data = msg.body
le = preprocessing.LabelEncoder()
#df1=pd.read_csv(StringIO(msg))
df1.drop(['Loan_ID', 'Application_Type'], axis=1, inplace=True)

df1['Gender']=df1['Gender'].fillna("Female")
le.fit(df1['Gender'].to_list())
df1['Gender']=le.transform(df1['Gender'])

df1['Married']=df1['Married'].fillna("Yes")
le.fit(df1['Married'].to_list())
df1['Married']=le.transform(df1['Married'])

le.fit(df1['Education'].to_list())
df1['Education']=le.transform(df1['Education'])

df1['Self_Employed']=df1['Self_Employed'].fillna("Yes")
le.fit(df1['Self_Employed'].to_list())
df1['Self_Employed']=le.transform(df1['Self_Employed'])

df1['Property_Area']=df1['Property_Area'].fillna("Urban")
le.fit(df1['Property_Area'].to_list())
df1['Property_Area']=le.transform(df1['Property_Area'])

df1['Dependents']=df1['Dependents'].fillna(0)
df1['Dependents']=df1['Dependents'].replace("3+", 3)

df1['Loan_Status']=df1['Loan_Status'].fillna("Y")
le.fit(df1['Loan_Status'].to_list())
df1['Loan_Status']=le.transform(df1['Loan_Status'])

df1['ApplicantIncome']=df1['ApplicantIncome'].str.replace(',', '')

In [8]:
df1

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1,0,0,0,0,5849,0.0,146.412162,360,1.0,2,1
1,1,1,1,0,0,4583,1508.0,128.000000,360,1.0,0,0
2,1,1,0,0,1,3000,0.0,66.000000,360,1.0,2,1
3,1,1,0,1,0,2583,2358.0,120.000000,360,1.0,2,1
4,1,0,0,0,0,6000,0.0,141.000000,360,1.0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
618,0,0,0,0,1,4583,0.0,60.000000,360,1.0,2,0
619,0,0,0,0,1,4563,0.0,150.000000,360,0.0,2,0
620,0,0,0,0,1,4583,0.0,105.000000,360,1.0,2,0
621,0,0,0,0,1,5583,0.0,405.000000,360,0.0,2,0


In [9]:
X = df1.iloc[:,0:11]
y = df1.iloc[:,11]

In [10]:
## split into Train and Testing for scoring the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False) # 80% training and 20% test

In [11]:
#build Model
#model_sarimax = SARIMAX(train_data['GROSS_SALES_QTY'],order =(0,1,0), seasonal_order=(0,1,0,24))
model_randomnforest=RandomForestClassifier(n_estimators=100)
model_randomnforest.fit(X_train,y_train)

RandomForestClassifier()

In [12]:
#Score the model
y_pred = model_randomnforest.predict(X_test)
Accuracy=metrics.accuracy_score(y_test, y_pred)
#rms_error =np.sqrt(mean_squared_error(test_data_val, df1['forecast'].dropna()))
accu=str(Accuracy)

In [13]:
print(accu)

0.784


In [14]:
##output model Blob
#pickled_clf = pickle.dumps(model_randomnforest)
#model_blob=pickled_clf

In [15]:
import pickle
pickle.dump(model_randomnforest, open("loan_rf.pickle.dat", "wb"))

In [16]:
feat_importances = pd.Series(model_randomnforest.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')

<AxesSubplot:>

In [ ]:
    # send the metrics to the output port - Submit Metrics operator will use this to persist the metrics
    metrics_dict = {"Accuray of Prediction": accu}
    api.send("metrics", api.Message(metrics_dict))

In [ ]:
    # create & send the model blob to the output port - Artifact Producer operator will use this to persist the model and create an artifact ID
    #model_blob = bytes("example", 'utf-8')
    api.send("modelBlob", model_blob)